# 1.Preprocess validation and test set_Calculate Metrics
This notebook reads in the csv/excel files with the unprocessed text, split it into sentences and maps it to the labels coming from INCEpTION. In addition to this, calculates some metrics to evaluate the performance of the annotators.

In [ ]:
# load packages
import pandas as pd
import re
import nltk
import copy
import numpy as np
from fuzzywuzzy import fuzz

nltk.download('punkt')
nltk.download('punkt_tab')
tokenizer=nltk.data.load('tokenizers/punkt/dutch.pickle')
extra_abbreviations = ['Dr', 'Mvw', 'Conclusie', 'Speciële anamnese','Familieanamnese','Beleid']
tokenizer._params.abbrev_types.update(extra_abbreviations)

In [ ]:
round_nr='first' #To be repeated for each round of annotation

In [ ]:
##Read data coming from INCEpTION (https://inception-project.github.io), coming from Annotator 1 and Annotator 2
## Must be performed for all rounds of annotations, for this project we had 3 rounds.
#Sample_selec contains all notes selected for annotations, including the note number, patient pseudo ID and the report type (e.g. multi-disciplinary meeting, pre-operative, family consult etc.)

#annotator1=pd.read_csv('')
#annotator1=pd.read_csv('')
#ann1['annotator'] = 'annotator1'
#ann2['annotator'] = 'annotator2'
#annotations = pd.concat([ann1, ann2], ignore_index=True)

#sample_selec=pd.read.csv('')  

In [ ]:
##This function was useful in our data, where the note number was written in the text. Can be ignored

def extract_note_nr(text):
    matches = re.findall(r'\d+', text)
    return matches[-1] if matches else None

annotations['note_nr'] = annotations['source_file'].apply(extract_note_nr)
annotations['note_nr'] = annotations['note_nr'].astype(int)


# fix any NaN-mistakes
annotations = annotations.dropna(subset=['note_nr'])
sample_selec = sample_selec.dropna(subset=['note_nr'])

# set all note_nrs to same type for consistency
annotations['note_nr'] = annotations['note_nr'].astype(int)
sample_selec['note_nr'] = sample_selec['note_nr'].astype(int)
sample_selec1['note_nr'] = sample_selec1['note_nr'].astype(int)
sample_selec2['note_nr'] = sample_selec2['note_nr'].astype(int)

In [ ]:
# Create new '_sentence_text' column and fill with snippets (snippets selected by annotators)

#INCEpTION has a different sentence splitting method. This function helps to assign to each sentence the right label by getting a snippet of text around each sentence.
def get_snippet(row):
    note_nr = row['note_nr']
    begin = int(row['begin']) - 73  #this offset must be set specifically for your dataset
    end = int(row['end']) -30 #this offset must be set specifically for your dataset
    #print(row)
    # Look up the note number in sample_selec
    censored_text = sample_selec[sample_selec['note_nr'] == note_nr]['censored'].values[0]
    
    # Get the snippet from the censored text
    snippet = censored_text[begin:end]
    
    return snippet

# Rename '_sentence_text' to 'inception_sentence'
annotations.rename(columns={'_sentence_text': 'inception_sentence'}, inplace=True)
annotations['_sentence_text'] = annotations.apply(get_snippet, axis=1)


In [ ]:
# match snippets from annotators with original sentences from dataset
sample_selec['sentences'] = sample_selec['censored'].apply(lambda x: tokenizer.tokenize(x))

In [ ]:
# distinguish annotators
annotator1 = annotations[annotations['annotator'] == 'annotator1']
annotator2 = annotations[annotations['annotator'] == 'annotator2']

In [ ]:
# match snippets annotators to original dataset
def find_match(annotator1, note_nr, sentence, sentence_context, sentences, i, threshold=90):
    def is_fuzzy_match(text1, text2, threshold=90):
        return fuzz.partial_ratio(text1, text2) >= threshold
    
    match = annotator1[(annotator1['note_nr'] == note_nr) & 
                       (annotator1['_sentence_text'].apply(lambda x: is_fuzzy_match(x, sentence)) |
                        annotator1['_sentence_text'].apply(lambda x: is_fuzzy_match(x, sentence_context)) |
                        annotator1['_sentence_text'].apply(lambda x: is_fuzzy_match(sentence, x)) |
                        annotator1['_sentence_text'].apply(lambda x: (
                            is_fuzzy_match(x[:10], sentence) and (i-1 >= 0 and is_fuzzy_match(x[10:], sentences[i-1]))
                        ) or (
                            is_fuzzy_match(x[-10:], sentence) and (i+1 < len(sentences) and is_fuzzy_match(x[:-10], sentences[i+1]))
                        )))]

    return match

In [ ]:
#create labels for annotator1


context_flag=1

# Assuming sample_selec and annotator1 are already defined DataFrames

# Copy sample_selec to annotator1_results
annotator1_results = copy.deepcopy(sample_selec)
annotator1_note_nrs = list(sample_selec1['note_nr'])
annotator1_results = annotator1_results[annotator1_results['note_nr'].isin(annotator1_note_nrs)]


# Create a new column 'manual_sentence_labels' by copying the 'sentences' column
annotator1_results['manual_sentence_labels'] = annotator1_results['sentences'].copy()
annotator1_results['context_sentences']=annotator1_results['sentences'].copy

# Iterate over the rows in annotator1_results
for idx, row in annotator1_results.iterrows():
    print(idx)
    note_nr = row['note_nr']
    for i in range(len(row.sentences)):
        sentences2=[]
        if context_flag:
        # Check if sentences[i-1] exists
            if i - 1 >= 0:
                context.append(sentences[i-1])
        # Add sentences[i] (it is assumed that sentences[i] always exists)
        context.append(sentences[i])

        if context_flag:
        # Check if sentences[i+1] exists
            if i + 1 < len(sentences):
                context.append(sentences[i+1])
        # Join the valid sentences with spaces
        sentence_context = '.'.join(context)
        sentences2.append(sentence_context)
    annotator1_results.loc[idx,'context_sentences']=sentences2
    
    if note_nr in annotator1['note_nr'].values:
        for i in range(len(row.sentences)):
            sentences2=[]
            if context_flag:
            # Check if sentences[i-1] exists
                if i - 1 >= 0:
                    context.append(sentences[i-1])
            # Add sentences[i] (it is assumed that sentences[i] always exists)
            context.append(sentences[i])

            if context_flag:
            # Check if sentences[i+1] exists
                if i + 1 < len(sentences):
                    context.append(sentences[i+1])
            # Join the valid sentences with spaces
            sentence_context = '.'.join(context)
            sentences2.append(sentence_context)
        annotator1_results.loc[idx,'context_sentences']=sentences2
        # Get the corresponding sentences for the note_nr
        sentences = row['context_sentences']#row['manual_sentence_labels']
        updated_sentences = []
#         for sentence in sentences:
#             print('sentence:', sentence)
#             match = annotator1[(annotator1['note_nr'] == note_nr) & (annotator1['_sentence_text'] == sentence)]
#             if not match.empty:
#                 updated_sentences.append(match['WHOPS'].values[0])
#             else:
#                 updated_sentences.append(np.nan)


        for i in range(len(sentences)):
            sentence = sentences[i]
            # Initialize an empty list to hold the valid sentences
            context = []
            #if context_flag:
            # Check if sentences[i-1] exists
            #    if i - 1 >= 0:
            #        context.append(sentences[i-1])
            # Add sentences[i] (it is assumed that sentences[i] always exists)
            context.append(sentences[i])
            
            #if context_flag:
            # Check if sentences[i+1] exists
            #    if i + 1 < len(sentences):
            #        context.append(sentences[i+1])
            # Join the valid sentences with spaces
            sentence_context = '.'.join(context)
            #print('sentence:', sentence)
            #print('sentence_context', sentence_context)
            # Find the match using the function
            result = find_match(annotator1, note_nr, sentence, sentence_context, sentences, i)
            #sentences2.append(sentence_context)
            # Append the corresponding value or np.nan to updated_sentences
            if not result.empty:
                if result['WHOPS'].values[0] == 'None':
                    updated_sentences.append(np.nan)
                else:
                    updated_sentences.append(result['WHOPS'].values[0])
            else:
                updated_sentences.append(np.nan)
        # Update the 'manual_sentence_labels' column with the new values
        #annotator1_results.at[idx,'sentences']=sentences2
        annotator1_results.at[idx, 'manual_sentence_labels'] = updated_sentences
        
# Iterate over the rows in annotator1_results again to update non-annotated sentences
for idx, row in annotator1_results.iterrows():
    note_nr = row['note_nr']
    sentences = row['manual_sentence_labels']
    if all(isinstance(s, str) for s in sentences):
        match = annotator1[annotator1['note_nr'] == note_nr]
        if match.empty:
            annotator1_results.at[idx, 'manual_sentence_labels'] = [np.nan] * len(sentences)

# Update 'None' labels to np.nan in 'manual_sentence_labels'
annotator1_results['manual_sentence_labels'] = annotator1_results['manual_sentence_labels'].apply(
    lambda x: [np.nan if v is None else v for v in x]
)

# Replace 'WHO 0', 'WHO 1', etc. with the corresponding integer number in 'manual_sentence_labels'
annotator1_results['manual_sentence_labels'] = annotator1_results['manual_sentence_labels'].apply(
    lambda x: [int(v.split()[1]) if isinstance(v, str) and v.startswith("WHO") else v for v in x]
)

# Copy 'manual_sentence_labels' to 'relevance_manual' and replace values
annotator1_results['relevance_manual'] = annotator1_results['manual_sentence_labels'].apply(
    lambda x: [0 if isinstance(v, float) and np.isnan(v) else 1 for v in x]
)









annotator1_results['note_PS_manual'] = np.nan

# Iterate over the rows in annotator1_results
for idx, row in annotator1_results.iterrows():
    note_nr = row['note_nr']
    # Look up the rows in annotator1 with the same note_nr
    matches = annotator1[annotator1['note_nr'] == note_nr]
    # Check if any of these rows contain 'report_type' in the 'sentence_text' column
    report_type_match = matches[matches['inception_sentence'].str.contains('report_type', na=False)]
    if not report_type_match.empty:
        # Assign the value from the 'WHOPS' column of the first matching row to 'note_PS_manual'
        annotator1_results.at[idx, 'note_PS_manual'] = report_type_match['WHOPS'].values[0]

        
        
# Convert 'WHO 0', 'WHO 1', etc. in 'note_PS_manual' to their corresponding integers and ensure NaN values are np.nan
annotator1_results['note_PS_manual'] = annotator1_results['note_PS_manual'].apply(
    lambda x: int(x.split()[1]) if isinstance(x, str) and x.startswith("WHO") else np.nan
)





# Copy the 'note_PS_manual' column and replace values
annotator1_results['relevance_PS_manual'] = annotator1_results['note_PS_manual'].apply(
    lambda x: 0 if np.isnan(x) else 1
)




# Create the new column 'previous_ann' initialized with 'no'
annotator1_results['previous_ann'] = 0

# Iterate over the rows in annotator1_results
for idx, row in annotator1_results.iterrows():
    note_nr = row['note_nr']
    # Look up the rows in annotator1 with the same note_nr
    matches = annotator1[annotator1['note_nr'] == note_nr]
    # Check if any of these rows contain 'WHO PS already annotated' in the 'WHOPScat' column
    if any(matches['WHOPScat'].str.contains('WHO PS already annotated', na=False)):
        annotator1_results.at[idx, 'previous_ann'] = 1


In [ ]:
# create labels for annotator2

# Assuming sample_selec and annotator2 are already defined DataFrames

# Copy sample_selec to annotator2_results
annotator2_results = copy.deepcopy(sample_selec)
annotator2_note_nrs = list(sample_selec2['note_nr'])
annotator2_results = annotator2_results[annotator2_results['note_nr'].isin(annotator2_note_nrs)]

# Create a new column 'manual_sentence_labels' by copying the 'sentences' column
annotator2_results['manual_sentence_labels'] = annotator2_results['sentences'].copy()

# Iterate over the rows in annotator2_results
for idx, row in annotator2_results.iterrows():
    note_nr = row['note_nr']
    if note_nr in annotator2['note_nr'].values:
        # Get the corresponding sentences for the note_nr
        sentences = row['manual_sentence_labels']
        updated_sentences = []
#         for sentence in sentences:
#             print('sentence:', sentence)
#             match = annotator1[(annotator1['note_nr'] == note_nr) & (annotator1['_sentence_text'] == sentence)]
#             if not match.empty:
#                 updated_sentences.append(match['WHOPS'].values[0])
#             else:
#                 updated_sentences.append(np.nan)
        for i in range(len(sentences)):
            sentence = sentences[i]
            # Initialize an empty list to hold the valid sentences
            context = []
            if context_flag:
            # Check if sentences[i-1] exists
                if i - 1 >= 0:
                    context.append(sentences[i-1])
            # Add sentences[i] (it is assumed that sentences[i] always exists)
            context.append(sentences[i])
            
            if context_flag:
            # Check if sentences[i+1] exists
                if i + 1 < len(sentences):
                    context.append(sentences[i+1])
            # Join the valid sentences with spaces
            sentence_context = '.'.join(context)
            #print('sentence:', sentence)
            #print('sentence_context', sentence_context)
            # Find the match using the function
            result = find_match(annotator1, note_nr, sentence, sentence_context, sentences, i)
            sentences2.append(sentence_context)
            # Append the corresponding value or np.nan to updated_sentences
            if not result.empty:
                if result['WHOPS'].values[0] == 'None':
                    updated_sentences.append(np.nan)
                else:
                    updated_sentences.append(result['WHOPS'].values[0])
            else:
                updated_sentences.append(np.nan)
        # Update the 'manual_sentence_labels' column with the new values
        annotator2_results.at[idx,'sentences']=sentences2
        annotator2_results.at[idx, 'manual_sentence_labels'] = updated_sentences

# Iterate over the rows in annotator2_results again to update non-annotated sentences
for idx, row in annotator2_results.iterrows():
    note_nr = row['note_nr']
    sentences = row['manual_sentence_labels']
    if all(isinstance(s, str) for s in sentences):
        match = annotator2[annotator2['note_nr'] == note_nr]
        if match.empty:
            annotator2_results.at[idx, 'manual_sentence_labels'] = [np.nan] * len(sentences)

# Update 'None' labels to np.nan in 'manual_sentence_labels'
annotator2_results['manual_sentence_labels'] = annotator2_results['manual_sentence_labels'].apply(
    lambda x: [np.nan if v is None else v for v in x]
)

# Replace 'WHO 0', 'WHO 1', etc. with the corresponding integer number in 'manual_sentence_labels'
annotator2_results['manual_sentence_labels'] = annotator2_results['manual_sentence_labels'].apply(
    lambda x: [int(v.split()[1]) if isinstance(v, str) and v.startswith("WHO") else v for v in x]
)

# Copy 'manual_sentence_labels' to 'relevance_manual' and replace values
annotator2_results['relevance_manual'] = annotator2_results['manual_sentence_labels'].apply(
    lambda x: [0 if isinstance(v, float) and np.isnan(v) else 1 for v in x]
)









annotator2_results['note_PS_manual'] = np.nan

# Iterate over the rows in annotator2_results
for idx, row in annotator2_results.iterrows():
    note_nr = row['note_nr']
    # Look up the rows in annotator2 with the same note_nr
    matches = annotator2[annotator2['note_nr'] == note_nr]
    # Check if any of these rows contain 'report_type' in the 'sentence_text' column
    report_type_match = matches[matches['inception_sentence'].str.contains('report_type', na=False)]
    if not report_type_match.empty:
        # Assign the value from the 'WHOPS' column of the first matching row to 'note_PS_manual'
        annotator2_results.at[idx, 'note_PS_manual'] = report_type_match['WHOPS'].values[0]

        
        
# Convert 'WHO 0', 'WHO 1', etc. in 'note_PS_manual' to their corresponding integers and ensure NaN values are np.nan
annotator2_results['note_PS_manual'] = annotator2_results['note_PS_manual'].apply(
    lambda x: int(x.split()[1]) if isinstance(x, str) and x.startswith("WHO") else np.nan
)





# Copy the 'note_PS_manual' column and replace values
annotator2_results['relevance_PS_manual'] = annotator2_results['note_PS_manual'].apply(
    lambda x: 0 if np.isnan(x) else 1
)




# Create the new column 'previous_ann' initialized with 'no'
annotator2_results['previous_ann'] = 0

# Iterate over the rows in annotator2_results
for idx, row in annotator2_results.iterrows():
    note_nr = row['note_nr']
    # Look up the rows in annotator1 with the same note_nr
    matches = annotator2[annotator2['note_nr'] == note_nr]
    # Check if any of these rows contain 'WHO PS already annotated' in the 'WHOPScat' column
    if any(matches['WHOPScat'].str.contains('WHO PS already annotated', na=False)):
        annotator2_results.at[idx, 'previous_ann'] = 1



# Select overlapping files

In [ ]:
# make backup of original
annotator1_results_all = copy.deepcopy(annotator1_results)
annotator2_results_all = copy.deepcopy(annotator2_results)
sample_selec_all = copy.deepcopy(sample_selec)

# create overlapping file datasets
common_note_nrs = set(annotator1_results['note_nr']).intersection(annotator2_results['note_nr'])
annotator1_results = annotator1_results[annotator1_results['note_nr'].isin(common_note_nrs)]
annotator2_results = annotator2_results[annotator2_results['note_nr'].isin(common_note_nrs)]
sample_selec = sample_selec[sample_selec['note_nr'].isin(common_note_nrs)]

# delete any accidental double files
annotator1_results = annotator1_results.drop_duplicates(subset=['note_nr', 'censored'], keep='first')
annotator2_results = annotator2_results.drop_duplicates(subset=['note_nr', 'censored'], keep='first')


annotator1_results = annotator1_results.reset_index(drop=True)
annotator2_results = annotator2_results.reset_index(drop=True)


In [ ]:
# check wether results annotators are aligned (brief overview)
for col in ['manual_sentence_labels', 'relevance_manual', 'note_PS_manual', 'relevance_PS_manual', 'previous_ann']:
    print('Col:', col)
    for i in range(len(annotator1_results)):
        if annotator1_results[col][i] != annotator2_results[col][i]:
            print('note_nr:', annotator1_results['note_nr'][i])
            print('ann1:', annotator1_results[col][i])
            print('ann2:', annotator2_results[col][i])

# Metrics for sentence level classification task

In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score

# Flatten the lists into one long list for each annotator
labels_annotator1_flat = [item for sublist in annotator1_results['relevance_manual'] for item in sublist]
labels_annotator2_flat = [item for sublist in annotator2_results['relevance_manual'] for item in sublist]

# Convert to numerical values
#labels_annotator1_num = [1 if x == 'relevant' else 0 for x in labels_annotator1_flat]
#labels_annotator2_num = [1 if x == 'relevant' else 0 for x in labels_annotator2_flat]

labels_annotator1_num = labels_annotator1_flat
labels_annotator2_num = labels_annotator2_flat

# Check the converted labels
print("Labels Annotator 1 (numerical):", labels_annotator1_num)
print("Labels Annotator 2 (numerical):", labels_annotator2_num)

# Function to calculate precision, recall, and F1-score
def calculate_metrics(true_labels, predicted_labels):
    precision = precision_score(true_labels, predicted_labels, zero_division=0)
    recall = recall_score(true_labels, predicted_labels, zero_division=0)
    f1 = f1_score(true_labels, predicted_labels, zero_division=0)
    return precision, recall, f1

# Annotator 1 as ground truth
precision_a1, recall_a1, f1_a1 = calculate_metrics(labels_annotator1_num, labels_annotator2_num)
print(f"Annotator 1 as ground truth -> Precision: {precision_a1}, Recall: {recall_a1}, F1-Score: {f1_a1}")

# Annotator 2 as ground truth
precision_a2, recall_a2, f1_a2 = calculate_metrics(labels_annotator2_num, labels_annotator1_num)
print(f"Annotator 2 as ground truth -> Precision: {precision_a2}, Recall: {recall_a2}, F1-Score: {f1_a2}")

# Calculate Cohen's Kappa
kappa = cohen_kappa_score(labels_annotator1_num, labels_annotator2_num)
print(f"Cohen's Kappa: {kappa}")
print()
print(f"Total sentences: {len(labels_annotator1_num)}")
print(f"Total PS ann1: {labels_annotator1_num.count(1)}")
print(f"Total NaN ann1: {labels_annotator1_num.count(0)}")
print(f"Total PS ann2: {labels_annotator2_num.count(1)}")
print(f"Total NaN ann2: {labels_annotator2_num.count(0)}")


# Metrics for sentence level regression task

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import krippendorff

# Sample data (you can replace this with your actual data)
# annotator1_results = pd.DataFrame({'manual_sentence_labels': [[np.nan, 0, 1, np.nan, 2], [1, np.nan, 2, 3, 4]]})
# annotator2_results = pd.DataFrame({'manual_sentence_labels': [[1, 0, np.nan, 2, 2], [np.nan, 1, 2, np.nan, 4]]})

# Extract the columns
#annotations1 = annotator1_results['manual_sentence_labels']
#annotations2 = annotator2_results['manual_sentence_labels']

#print('annotations1:', annotations1)
#print('annotations2:', annotations2)

# Flatten the lists into a single list and replace 'None' and any other non-numeric with np.nan
flatten_annotations1 = [item for sublist in annotator1_results['manual_sentence_labels'] for item in sublist]
flatten_annotations2 = [item for sublist in annotator2_results['manual_sentence_labels'] for item in sublist]


# Convert to pandas Series
annotations1_series = pd.Series(flatten_annotations1, dtype=float)
annotations2_series = pd.Series(flatten_annotations2, dtype=float)

#print('Flattened and cleaned annotations1:', annotations1_series)
#print('Flattened and cleaned annotations2:', annotations2_series)

# Print the number of NaNs in each series
print('Number of NaNs in annotations1_series:', annotations1_series.isna().sum())
print('Number of NaNs in annotations2_series:', annotations2_series.isna().sum())

# Drop rows where either annotator has a NaN value for MAE calculation
valid_indices_mae = annotations1_series.notna() & annotations2_series.notna()
valid_annotations1_mae = annotations1_series[valid_indices_mae]
valid_annotations2_mae = annotations2_series[valid_indices_mae]

print('Valid annotations for MAE - Annotator 1:', valid_annotations1_mae)
print('Valid annotations for MAE - Annotator 2:', valid_annotations2_mae)

if len(valid_annotations1_mae) > 0 and len(valid_annotations2_mae) > 0:
    # Calculate MAE assuming annotator 1 as the gold truth
    mae_ann1_as_truth = mean_absolute_error(valid_annotations1_mae, valid_annotations2_mae)
    
    # Calculate MAE assuming annotator 2 as the gold truth
    mae_ann2_as_truth = mean_absolute_error(valid_annotations2_mae, valid_annotations1_mae)
    
    # Print the MAE results
    print("MAE (Annotator 1 as truth):", mae_ann1_as_truth)
    print("MAE (Annotator 2 as truth):", mae_ann2_as_truth)
else:
    print("Insufficient valid data for MAE calculation.")

# Drop NaN values for Krippendorff's Alpha calculation
valid_indices_ka = annotations1_series.notna() & annotations2_series.notna()
valid_annotations1_ka = annotations1_series[valid_indices_ka]
valid_annotations2_ka = annotations2_series[valid_indices_ka]

print('Valid annotations for Krippendorff\'s Alpha - Annotator 1:', valid_annotations1_ka)
print('Valid annotations for Krippendorff\'s Alpha - Annotator 2:', valid_annotations2_ka)

# Check for sufficient variability and valid data
if len(valid_annotations1_ka) > 0 and len(valid_annotations2_ka) > 0 and valid_annotations1_ka.nunique() > 1 and valid_annotations2_ka.nunique() > 1:
    # Create an array for Krippendorff's Alpha
    annotations_list = np.array([valid_annotations1_ka.values, valid_annotations2_ka.values])
    
    # Calculate Krippendorff's Alpha
    alpha = krippendorff.alpha(reliability_data=annotations_list, level_of_measurement='nominal')
    
    # Print Krippendorff's Alpha
    print("Krippendorff's Alpha:", alpha)
else:
    print("Insufficient variability or valid data for Krippendorff's Alpha calculation.")


In [ ]:
annotator1_results = annotator1_results.reset_index(drop=True)
annotator2_results = annotator2_results.reset_index(drop=True)

# check wether results annotators are aligned (brief overview)
for col in ['manual_sentence_labels', 'relevance_manual', 'note_PS_manual', 'relevance_PS_manual', 'previous_ann']:
    print('Col:', col)
    for i in range(len(annotator1_results)):
        print('Row:', i)
        print('ann1:', annotator1_results[col][i])
        print('ann2:', annotator2_results[col][i])

# Write to csv

In [ ]:
# double files docs
annotator1_results['annotator'] = 'annotator1'
annotator1_results['round'] = round_nr
annotator2_results['annotator'] = 'annotator2'
annotator2_results['round'] = round_nr

# double files org
df_iaa = pd.concat([annotator1_results, annotator2_results])

In [ ]:
df_iaa.to_csv(f"./Intermediate results/{round_nr}.csv", sep="\t")